In [ ]:
'''
PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group 
the passenger is travelling with and pp is their number within the group. People in a group are often family members, 
but not always.

HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.

CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the 
voyage. Passengers in cryosleep are confined to their cabins.

Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P 
for Port or S for Starboard.

Destination - The planet the passenger will be debarking to.

Age - The age of the passenger.

VIP - Whether the passenger has paid for special VIP service during the voyage.

RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship 
Titanic's many luxury amenities.

Name - The first and last names of the passenger.

Transported - Whether the passenger was transported to another dimension. This is the target, the column you are 
trying to predict.
'''

In [1]:
# Import Dependencies for Download and Clean
# run in terminal to use bigquery  "pip install --upgrade google-cloud-bigquery"
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file(
'/Users/spicious/Desktop/News Datasets/Resources/spaceship-titanic-387720-729aac731f9f.json')

In [ ]:
# Import Dependencies for Random Forest Model
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Import for Data Imputation
from scipy import stats
import numpy as np
from sklearn import impute

In [ ]:
project_id = 'spaceship-titanic-387720'
client = bigquery.Client(credentials= credentials,project=project_id)

In [ ]:
query_Amenities = client.query("""
   SELECT *
   FROM Starship_Titanic.Amenities""")

query_PassengerInfo = client.query("""
   SELECT *
   FROM Starship_Titanic.PassengerInfo""")

query_PlanetInfo = client.query("""
   SELECT *
   FROM Starship_Titanic.Planet""")

results_Amenities = query_Amenities.result()
results_PassengerInfo = query_PassengerInfo.result()
results_PlanetInfo = query_PlanetInfo.result()

Amenities_df = pd.DataFrame(results_Amenities)
PassengerInfo_df = pd.DataFrame(results_PassengerInfo)
PlanetInfo_df = pd.DataFrame(results_PlanetInfo)

In [ ]:
print(Amenities_df.loc[0][0])
print(PassengerInfo_df.loc[0][0])
print(PlanetInfo_df.loc[0][0])

In [ ]:
Amenities_sorted = pd.DataFrame()
i = 0
while i < 8:
    Amenities_sorted[i] = Amenities_df[0].apply(lambda x: x[i])
    i += 1

Amenities_sorted = Amenities_sorted.rename(columns={0: "PassengerId", 
                   1: "Cabin", 
                   2: "VIP", 
                   3: "RoomService",
                   4: "FoodCourt",
                   5: "ShoppingMall",
                   6: "Spa",
                   7: "VRDeck"})

In [ ]:
PassengerInfo_sorted = pd.DataFrame()
i = 0
while i < 6:
    PassengerInfo_sorted[i] = PassengerInfo_df[0].apply(lambda x: x[i])
    i += 1
    
PassengerInfo_sorted = PassengerInfo_sorted.rename(columns={0: "PassengerId", 
                   1: "Name", 
                   2: "HomePlanet", 
                   3: "Cabin",
                   4: "Age",
                   5: "Transported"})

In [ ]:
PlanetInfo_sorted = pd.DataFrame()
i = 0
while i < 4:
    PlanetInfo_sorted[i] = PlanetInfo_df[0].apply(lambda x: x[i])
    i += 1
    
PlanetInfo_sorted = PlanetInfo_sorted.rename(columns={0: "PassengerId", 
                   1: "HomePlanet", 
                   2: "Destination", 
                   3: "CryoSleep"})

In [ ]:
# Merging tables
merge_df = pd.merge(Amenities_sorted, PassengerInfo_sorted, on = 'PassengerId', how = 'inner')
df = pd.merge(merge_df, PlanetInfo_sorted, on = 'PassengerId', how = 'inner')

In [ ]:
# Cleaning up Column Duplications
df = df.drop(['Cabin_y', 'HomePlanet_y'], axis=1)

In [ ]:
# Renaming Columns
df = df.rename(columns={'Cabin_x': "Cabin", 
                   'HomePlanet_x': "HomePlanet"})

In [ ]:
df["Transported"].value_counts()

In [ ]:
# Drop all rows with missing null's in the training data column
df = df["Transported"].dropna()

In [ ]:
#check dataframe
print(df.shape)
df.head(5)

In [ ]:
# Check columns
columns = list(df.columns)
columns

# Data Munging

In [ ]:
# Make plans
'''
1) Check for missing values in all columns
1.5) Consider Imputation
2) remove unnecessary columns such as PassengerId and Name
3) Convert categorical variables into indicator variables for HomePlanet, CryoSleep, Deck, Side, Destination, VIP, Transported
4) Split up Cabin column into three different features
'''

In [ ]:
# Split Cabin column in three different Columns
df[['Deck','RoomNum','Side']] = df.Cabin.str.split("/", expand = True)
del df['Cabin']

# Reorder dataset to make me happy
df = df[['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Deck',
 'RoomNum',
 'Side',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported']]

# Check to see everything is going to plan
print(df.head())

In [ ]:
# Drop unnecessary columns
del df['PassengerId']
del df['Name']

In [ ]:
# Check drops
columns = list(df.columns)
columns

In [ ]:
# examining missing values
print("Missing values distribution: ")
print(df.isnull().mean())

In [ ]:
# check datatype in each column
print("Column datatypes: ")
print(df.dtypes)

In [ ]:
# Exploring the data
print('HomePlanet', df["HomePlanet"].unique())
print('CryoSleep', df["CryoSleep"].unique())
print('Deck', df["Deck"].unique())
print('Side', df["Side"].unique())
print('Destination', df["Destination"].unique())
print('VIP', df["VIP"].unique())
print('Transported', df["Transported"].unique())

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df["Transported"].value_counts()

In [ ]:
df["Transported"].value_counts()

# Dealing with NaNs

In [ ]:
df_clean = df.dropna()
df_clean

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='titanic.csv')  
df.to_csv('titanic.zip', index=False,
          compression=compression_opts)  

# Random Forest Model

In [ ]:
# Define features set
X = df_clean.copy()
X.drop("Transported", axis=1, inplace=True)
X.head()

In [ ]:
# Define target vector
y = df_clean["Transported"].values.reshape(-1, 1)
y[:5]

In [ ]:
# Dummy Categorical Variables
X = pd.get_dummies(X)

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [ ]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [ ]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [ ]:
# Fit the model and use .ravel()on the "y_train" data. 
rf_model = rf_model.fit(X_train, y_train)

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test)

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Get the feature importance array
importances = rf_model.feature_importances_
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:100]

# Trying Data Imputation

In [ ]:
# Impute int values with mean.

titanic_df_impute = df.copy()

mean_imputer = impute.SimpleImputer(strategy='mean')

titanic_df_impute['Age'] = mean_imputer.fit_transform(titanic_df_impute['Age'].values.reshape(-1,1))
titanic_df_impute['RoomService'] = mean_imputer.fit_transform(titanic_df_impute['RoomService'].values.reshape(-1,1))
titanic_df_impute['FoodCourt'] = mean_imputer.fit_transform(titanic_df_impute['FoodCourt'].values.reshape(-1,1))
titanic_df_impute['ShoppingMall'] = mean_imputer.fit_transform(titanic_df_impute['ShoppingMall'].values.reshape(-1,1))
titanic_df_impute['Spa'] = mean_imputer.fit_transform(titanic_df_impute['Spa'].values.reshape(-1,1))
titanic_df_impute['VRDeck'] = mean_imputer.fit_transform(titanic_df_impute['VRDeck'].values.reshape(-1,1))

In [ ]:
# Check to see if Mean Imputation was performed
titanic_df_impute.isnull().sum()

In [ ]:
## Use ffil and fillna to fill object values with the value nearest to it
titanic_df_impute['RoomNum'] = titanic_df_impute['RoomNum'].fillna(method='ffill')
titanic_df_impute['HomePlanet'] = titanic_df_impute['HomePlanet'].fillna(method='ffill')
titanic_df_impute['CryoSleep'] = titanic_df_impute['CryoSleep'].fillna(method='ffill')
titanic_df_impute['Deck'] = titanic_df_impute['Deck'].fillna(method='ffill')
titanic_df_impute['Side'] = titanic_df_impute['Side'].fillna(method='ffill')
titanic_df_impute['VIP'] = titanic_df_impute['VIP'].fillna(method='ffill')
titanic_df_impute['Destination'] = titanic_df_impute['Destination'].fillna(method='ffill')

In [ ]:
# Check to see if categorical Imputation was performed
titanic_df_impute.isnull().sum()

In [ ]:
titanic_df_impute